In [1]:
import pandas as pd
import numpy as np
import os
import  matplotlib.pyplot as plt
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

In [148]:
country  = 'tanzania'
err_type = 'scaled_mse'
scale    = 1
path_to_results = './tanzania_v4_7_crop_sales/'
path_to_tables  = './Tables_7_v2/'

In [149]:
outcomes = sorted(set([x.split('___')[0] for x in os.listdir(path_to_results) if 'coef' not in x and 'clus' not in x and 'plot' not in x and 'lock' not in x and 'elbow' not in x]))

model_names = {
'All variables':'All Inputs',
'Highly correlated policy and non-policy variables':'Relevant Inputs',
'Policy variables':'Policy Inputs',
}

index_row = []

In [150]:
outcomes

['crop_sales']

In [142]:

def summary_mse(year,outcome,var_spec):
    
    mse = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==False)][err_type].values[0]

    idx = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==True)].index#[err_type].idxmin()

    mse_segments = list(raw_mse.loc[idx][err_type])

    segments = list(raw_mse.loc[idx]['segment'].apply(lambda x: x.replace('___' + country + '_'+str(year), '')))

    all_series = pd.DataFrame()
    for segment, mse_segment in zip(segments, mse_segments):
        series = pd.Series([
            year,
            outcome.replace('_',' ').title(),
            'LASSO + '+model_names[var_spec],
            np.round(mse/scale,3),
            segment.replace('_',' ').title(),
            np.round(mse_segment/scale,3),
            np.round(mse/mse_segment-1,3)],
            index=[
            'Year',
            'Outcome',
            'Model',
            'Mean Sq. Error',
            'Household Segmentation',
            'Mean Sq. Error Segmented',
            '% Reduction in Error',
            ])
        all_series = pd.concat([all_series, series], 1)
    
    return all_series
        
            
def summary_coef(year,outcome,var_spec):
    global index_row
    # Get Segment Corresponding to Min MSE
    idx = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==True)].index#[err_type].idxmin()
    
    best_segments = list(raw_mse.loc[idx]['segment'].apply(lambda x: x.replace('___' + country + '_'+str(year), '')))

    all_series = pd.DataFrame()
    for best_segment in best_segments:
#         &(raw_coef['segmentation']==var_spec[:-2]
        best_var_all = raw_coef.loc[(raw_coef['inputs']==var_spec)].filter(
            regex='policy').iloc[0].astype(float).apply(np.abs).idxmax()

#     &(raw_coef['segmentation']==var_spec[:-2])
        best_coef_all = raw_coef.loc[(raw_coef['inputs']==var_spec)].filter(
            regex='policy').iloc[0].astype(float)[best_var_all]
        segments = int(math.pow(2, len(best_segment.split(','))))
        best_var = {}
        best_coef = {}
        best_of = 5
        for k in range(4):
#             try:
            best_c = raw_coef.loc[
            (raw_coef['inputs']==var_spec)&\
            (raw_coef['segmentation']==best_segment)&\
            (raw_coef['segment']==str(k))].filter(
            regex='policy').iloc[0].astype(float).apply(np.abs).nlargest(best_of)
#             best_t = best_c.apply(lambda x: x.index.tolist(), axis=1)
#             print (best_t.as_matrix())
#             best_coefs =  best_c.as_matrix()
            best_vars = best_c.index.tolist()
            best_coefs = raw_coef.loc[
            (raw_coef['inputs']==var_spec)&\
            (raw_coef['segmentation']==best_segment)&\
            (raw_coef['segment']==str(k))].iloc[0][best_vars].as_matrix()
            print (best_coefs)

            row = []
            index_row =[]
            for ix, t, i in zip(range(best_of), best_vars, best_coefs):
                row += [
                    t.replace('___ethiopia_'+str(year), '').replace('_',' ').title(),
                    np.round(i,3)
                ]
                index_row += ['Var ' + str(ix), 'Coef ' +  str(ix)]
#             except:
#                 print (var_spec, best_segment, k)
            series = pd.Series([
                year,
                outcome.replace('_',' ').title(),
                'LASSO + '+model_names[var_spec],
                best_var_all.replace('___ethiopia_'+str(year), '').replace('_',' ').title(),
                np.round(best_coef_all,3),
                best_segment.replace('___ethiopia_'+str(year), '').title()] + row,index=[
                'Year',
                'Outcome',
                'Model',
                'Most Relevant Input',
                'Coef. Most Relevant Input',
                'Household Segmentation']
                    + index_row)
            all_series = pd.concat([all_series, series], 1)
        
    return all_series
        

In [154]:
import math
all_mse  = pd.DataFrame()
all_coef = pd.DataFrame()
all_raw_mse = pd.DataFrame()

all_raw_coef = pd.DataFrame()
years = [2014]

for outcome in outcomes:
    
    for year in years:
        
        try:
            raw_mse = pd.read_csv(
            path_to_results+outcome+'___'+'output'+'___'+country+'_'+str(year)+'.csv',
            usecols=['clustered','input',err_type,'segment'])
        except:
            continue
        
        raw_mse['output'] = outcome
        all_raw_mse = pd.concat([all_raw_mse, raw_mse], 0)
        all_raw_mse['segment'] = all_raw_mse['segment'].fillna('').apply(lambda x: x.replace('___ethiopia_'+str(year), ''))
        raw_coef = pd.read_csv(
        path_to_results+'coef_'+outcome+'___'+'output'+'___'+country+'_'+str(year)+'.csv',index_col=0)
        
        # Variables On Which Segmentation is Performed
        raw_coef['segmentation'] = raw_coef['model'].apply(
        lambda x:x.replace('Lasso_','').replace('OrdinaryLeastSquares_','').replace('clustered_','').replace('segmented_','').replace('___' + country + '_'+str(year), '')[:-2])

        # Segment Index
        raw_coef['output'] = outcome
        raw_coef['year'] = year
        raw_coef['segment'] = raw_coef['model'].apply(
        lambda x:x.split('_')[-1])
        all_raw_coef = pd.concat([all_raw_coef, raw_coef], 0)

        for var_spec in sorted(model_names):
            try:
#             all_mse  = pd.concat([all_mse, summary_mse(year,outcome,var_spec)],1)
                all_coef = pd.concat([all_coef, summary_coef(year,outcome,var_spec)],1)
            except:
               continue
            
# across_mse = all_raw_mse.groupby(['input', 'output', 'segment'])[err_type].mean()
# across_mse = across_mse.to_frame().reset_index()
# across_mse['univariate_segment'] = across_mse['segment'].apply(lambda x: (x!= '') & (len(x.split(',')) == 1))

# best_vars = pd.DataFrame()
# for var_spec in sorted(model_names):
#     for outcome in outcomes:
#         q = across_mse.loc[
#             (across_mse['input']==var_spec) &\
#             (across_mse['output']==outcome) &\
#             (across_mse['univariate_segment']==True)][err_type].nsmallest(2)
#         best_seg = ','.join(list(across_mse.iloc[q.index.values]['segment']))
        
#         row = pd.Series([var_spec, outcome, best_seg], index=['input', 'output','segment'])
#         best_vars = pd.concat([best_vars, row], 1)

# across_coefs = pd.DataFrame()
# for year in years:
#     for index, row in best_vars.iterrows():
#         output = row['output']
#         segment = row['segment']
# #         .split(',')
# #         segment = [x + '___ethiopia_' + str(year) for x in segment]
# #         segment = ','.join(segment)
#         segments = int(math.pow(2, len(segment.split(','))))
#         for k in range(segments):
#             best_var_row = all_raw_coef.loc[
#             (all_raw_coef['inputs']=='All variables')&\
#             (all_raw_coef['segmentation']==segment)&\
#             (all_raw_coef['output']==output)&\
#             (all_raw_coef['segment']==str(k))].filter(
#             regex='policy').filter(regex=str(year)).iloc[0]
#             best_var_row['best'] = best_var_row.astype(float).apply(np.abs).idxmax()
#             best_var_row['best_coef'] = best_var_row[best_var_row['best']]
#             best_var_row['year'] = year
#             best_var_row['output'] = output
#             across_coefs = pd.concat([across_coefs, best_var_row], 0)

# best_vars.T.to_excel(path_to_tables+'mse_across.xlsx', engine='openpyxl')
# #across_coefs.to_excel(path_to_tables+'coef_across.xlsx', engine='openpyxl')

# all_mse = \
# all_mse.T.set_index(['Outcome','Year','Model'])[[
# 'Household Segmentation',
# 'Mean Sq. Error',
# 'Mean Sq. Error Segmented',
# '% Reduction in Error',
# ]].copy()

all_coef = all_coef.T.set_index(['Outcome','Year','Model'])[[
'Household Segmentation',
'Most Relevant Input', 
'Coef. Most Relevant Input',] + index_row].copy()

# all_mse.to_excel(path_to_tables+'mse_'+country+'.xlsx', engine='openpyxl')

all_coef.to_excel(path_to_tables+'coef_'+country+'.xlsx', engine='openpyxl')

# all_mse.drop('Household Segmentation',1).astype(float).groupby('Model').mean().round(3).sort_values(
# by='% Reduction in Error').to_excel(path_to_tables+'summary_mse_'+country+'.xlsx', engine='openpyxl')

[2.1713266331882322 0.23353430932242805 -0.16598462653262766
 0.13517242830914086 -0.09012666869926067]
[2.1177599104128397 0.32953704878287265 -0.1625907605491691
 -0.14134324463635894 -0.11178101914556557]
[0.79504218553651018 0.2247067825395645 -0.2194437493815552
 0.18734639842300976 -0.086424851201796993]
[0.220573631237278 0.17524704359656268 -0.11568944634841415
 0.095904927919206406 -0.093444005897394003]
[0.17252044507406036 0.045077354285899401 0.009221137402644285]
[0.20616529317049592 0.085443765831932014 -0.084029602068004527]
[0.20658794105592326 0.16546331138292233 0.1252168372176288]
[0.11822685643224905 -0.031382673538666403 -0.013769918177501259]
[2.6517086728842845 0.23853270382695849 0.15000207704967347
 -0.14988884957093121 -0.1307493271789712]
[2.9137094401940771 0.27648576588323748 -0.19575734545983745
 -0.17934691440755976 -0.15230710421250573]
[0.46542459383509771 0.25244524744730562 0.22154712940124041
 0.14289588643165446 -0.13673513129043208]
[0.224825501677

,amount_of_assistance_received___policy___ethiopia_2015,attended_school___ethiopia_2015,average_precipitation___ethiopia_2015,average_temperature___ethiopia_2015,crop_diversification___policy___ethiopia_2015,distance_to_market___ethiopia_2015,distance_to_population_center___ethiopia_2015,distance_to_road___ethiopia_2015,elevation___ethiopia_2015,has_borrowed___policy___ethiopia_2015,...,rooting_conditions_:_very_severe_constraint___ethiopia_2015,rural_household___ethiopia_2015,uses_credit___policy___ethiopia_2015,uses_extension_program___policy___ethiopia_2015,uses_irrigation___policy___ethiopia_2015,variations_in_greenness___ethiopia_2015,segmentation,output,year,segment
0,-0.000000,0.000000,-0.159949,0.000000,-0.000000,-0.000000,0.065256,0.021202,-0.248077,0.026070,...,2.889459e-02,0.000000,-0.000000,0.000000,-0.043549,0.026026,"number_of_hired_workers___policy,land_surface,...",crop_sales,2015,0
1,-0.092434,-0.074096,-0.000000,-0.188754,0.000000,-0.102884,0.014142,-0.003876,-0.254845,-0.037707,...,0.000000e+00,0.020026,0.000000,-0.161246,0.085374,0.000000,"number_of_hired_workers___policy,land_surface,...",crop_sales,2015,1
2,0.032709,0.017542,-0.009003,0.000000,-0.077955,-0.098858,-0.042677,0.000000,-0.109006,-0.003745,...,0.000000e+00,-0.000196,-0.060645,-0.030416,0.043728,0.000000,"number_of_hired_workers___policy,land_surface,...",crop_sales,2015,2
3,-0.000000,0.000000,0.000000,0.000000,0.109882,-0.028723,-0.189510,-0.038913,-0.224639,-0.000000,...,-7.015432e-02,0.027892,0.031411,0.000000,0.006755,-0.197848,"number_of_hired_workers___policy,land_surface,...",crop_sales,2015,3
4,-0.002864,0.026827,-0.230404,-0.109365,-0.002087,-0.007952,0.087934,0.037934,-0.362289,0.034241,...,-2.149939e+10,0.002408,-0.000392,0.025737,-0.079044,0.129318,"number_of_hired_workers___policy,land_surface,...",crop_sales,2015,0
5,-0.126497,-0.209921,0.017348,-0.619707,0.009054,-0.081580,0.081888,-0.053639,-0.661126,-0.074816,...,-2.879487e+11,0.019092,-0.020619,-0.227136,0.128980,0.031091,"number_of_hired_workers___policy,land_surface,...",crop_sales,2015,1
6,0.038378,0.017320,-0.034330,0.040344,-0.090201,-0.081606,-0.077265,0.039080,-0.080494,-0.001720,...,-2.824002e+12,-0.011288,-0.086952,-0.067032,0.073896,0.077810,"number_of_hired_workers___policy,land_surface,...",crop_sales,2015,2
7,0.004268,0.012952,0.130400,-0.199563,0.126831,-0.009048,-0.166260,-0.056612,-0.475427,-0.019285,...,-2.432886e+11,0.052152,0.060400,0.010901,0.042043,-0.332260,"number_of_hired_workers___policy,land_surface,...",crop_sales,2015,3
8,0.030626,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.065649,...,NaN,NaN,-0.051949,-0.000000,0.052304,NaN,"number_of_hired_workers___policy,land_surface,...",crop_sales,2015,0
9,0.119615,NaN,NaN,NaN,-0.129489,NaN,NaN,NaN,NaN,-0.031369,...,NaN,NaN,-0.054353,-0.058762,0.170613,NaN,"number_of_hired_workers___policy,land_surface,...",crop_sales,2015,1


In [274]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
for year in ['2011', '2013', '2015']:
    for ctype in ['cluster', 'segment']:
        cluster_indices = pd.read_csv('./segment-cluster-compare/' + year + '_' + ctype + '.csv')
        plt.scatter(cluster_indices['precipitation___ethiopia_'+ year], cluster_indices['greenness___ethiopia_' + year], c=cluster_indices['cluster'], cmap=cm.tab10)
        plt.xlabel('Precipitation (in z-score)')
        plt.ylabel('Variation in Greenness (in z-score)')
        plt.savefig('./segment-cluster-compare/' + year + '-' + ctype + '.pdf', bbox_inches = 'tight')
        plt.clf()

In [275]:
cluster_indices

,Unnamed: 0,cluster,precipitation___ethiopia_2015,greenness___ethiopia_2015
0,0,1,-1.342898,-0.002066
1,1,1,-1.340160,-0.002066
2,2,1,-1.293613,-0.002066
3,3,1,-1.301828,-0.002066
4,4,1,-1.293613,-0.002066
5,5,1,-1.301828,-0.002066
6,6,1,-0.888381,-0.002066
7,7,1,-0.888381,-0.002066
8,8,1,-0.893858,-0.002066
9,9,1,-0.907548,-0.002066


In [276]:
raw_mse.loc[3,'segment'].replace('___ethiopia_'+str(year), '')

'crop_diversification___policy'

In [277]:
raw_mse

,clustered,input,scaled_mse,segment,output
0,False,All variables,0.817908,NaN,no_food_deficiency
1,True,All variables,0.814405,use_credit___policy___ethiopia_2015,no_food_deficiency
2,True,All variables,0.830746,household_head_is_male___ethiopia_2015,no_food_deficiency
3,True,All variables,0.805308,crop_diversification___policy___ethiopia_2015,no_food_deficiency
4,True,All variables,0.861480,"use_credit___policy___ethiopia_2015,household_...",no_food_deficiency
5,True,All variables,0.846366,"use_credit___policy___ethiopia_2015,household_...",no_food_deficiency
6,True,All variables,0.803856,"use_credit___policy___ethiopia_2015,crop_diver...",no_food_deficiency
7,True,All variables,0.808638,"use_credit___policy___ethiopia_2015,crop_diver...",no_food_deficiency
8,True,All variables,0.832983,"household_head_is_male___ethiopia_2015,crop_di...",no_food_deficiency
9,True,All variables,0.837017,"household_head_is_male___ethiopia_2015,crop_di...",no_food_deficiency
